## 5.7　誤差逆伝播法の実装

In [1]:
import os
import sys
import numpy as np

In [2]:
os.getcwd()

'/Users/yuta.shimizu/Downloads/ML'

In [3]:
os.chdir('/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master/ch01')
sys.path.append(os.pardir)

In [4]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict
from dataset.mnist import load_mnist
# from ch05.two_layer_net import TwoLayerNet

### 5.7.1　ニューラルネットワークの学習の全体図

In [5]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
    
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 :
            t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

### 5.7.3　誤差逆伝播法の勾配確認

勾配を求める方法は、数値微分と誤差逆伝播法の2つがあり<br>
数値微分は、誤差逆伝播法の結果を確かめるために利用される。

In [6]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ' : ' + str(diff))

W1 : 4.816236055647198e-10
b1 : 2.9101796198988812e-09
W2 : 6.948078639077565e-09
b2 : 1.3952738492500894e-07


パラメータの初期化に np.random.randn を使用しているので、最終的な出力結果も一定ではなくなる。

### 5.7.4　誤差逆伝播を使った学習

In [10]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
print('train_size:', train_size)
print('batch_size:', batch_size)
print('iter_per_epoch:', iter_per_epoch)
print('iters_num:', iters_num)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
        
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train_acc:', round(train_acc, 3), 'test_acc:', round(test_acc,3))

train_size: 60000
batch_size: 100
iter_per_epoch: 600.0
iters_num: 10000
train_acc: 0.086 test_acc: 0.086
train_acc: 0.901 test_acc: 0.908
train_acc: 0.925 test_acc: 0.928
train_acc: 0.937 test_acc: 0.937
train_acc: 0.945 test_acc: 0.945
train_acc: 0.952 test_acc: 0.949
train_acc: 0.957 test_acc: 0.953
train_acc: 0.961 test_acc: 0.957
train_acc: 0.964 test_acc: 0.961
train_acc: 0.967 test_acc: 0.961
train_acc: 0.968 test_acc: 0.962
train_acc: 0.97 test_acc: 0.964
train_acc: 0.972 test_acc: 0.966
train_acc: 0.975 test_acc: 0.966
train_acc: 0.976 test_acc: 0.967
train_acc: 0.978 test_acc: 0.969
train_acc: 0.979 test_acc: 0.969


In [8]:
print('最大値:', max(10, 5))
print('余り:', 8 % 3)

最大値: 10
余り: 2
